In [ ]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 필요한 라이브러리 임포트
import pandas as pd
import glob
import os

# 3. Google Drive 내 작업 폴더 경로 설정
folder_path = '/content/drive/My Drive/회사업무/신규P값종량제/병합'  # 'your_folder'를 Google Drive의 실제 폴더명으로 변경하세요

# 4. 모든 파일 경로 가져오기
all_files = glob.glob(folder_path + "/*.xls")

# 5. 파일명에 따른 정렬 기준 설정
file_order = [
    "0. 채널별, 월", "1.기관별, 월",  # "기관별" 파일을 채널 다음으로 정렬
    "2.강북_월", "2.강남_월", "2.부산_월",
    "2.대구", "2.전남", "2.충남",
    "사원별"
]

# 6. 파일명 정렬 함수 정의
def get_file_priority(file_name):
    for i, keyword in enumerate(file_order):
        if keyword in file_name:
            return i
    return len(file_order)  # 키워드가 없으면 가장 뒤로 정렬

# 7. 파일명에 따라 정렬
all_files.sort(key=lambda x: get_file_priority(os.path.basename(x)))

# 8. 전체 데이터프레임 초기화
all_data = pd.DataFrame()

# 9. 특정 패턴을 포함하는 행을 필터링하기 위한 함수 정의 (A열에만 적용)
def filter_out_specific_gun_rows(df):
    remove_patterns = [r'8\s건$', r'12\s건$', r'6\s건$', r'9\s건$', r'7\s건$']
    pattern = '|'.join(remove_patterns)
    return df[~df.iloc[:, 0].astype(str).str.contains(pattern)]

# 10. 각 파일의 데이터를 순서대로 병합
for i, file in enumerate(all_files):
    try:
        # HTML 형식으로 파일 읽기
        df_list = pd.read_html(file)
        df = df_list[0]  # 첫 번째 테이블만 사용

        # 특정 "숫자 + 공백 + 건" 패턴을 포함하는 행 제거 (A열만)
        df = filter_out_specific_gun_rows(df)

        # 파일명 출력 및 데이터 미리보기
        if i == 0:
            print(f"{file} 데이터 미리보기:\n", df.head())
        elif i == len(all_files) - 1 or "2.기관별_" in file:
            df = df.iloc[1:]  # 헤더 제외
            print(f"{file} 데이터 미리보기:\n 헤더제외 ->", df.head())
        else:
            print(f"{file} 데이터 미리보기:\n", df.head())

        # 첫 번째 파일에서만 헤더를 사용하고, 이후 파일들은 헤더 없이 추가
        if all_data.empty:
            all_data = df
        else:
            # 기존 열들과 현재 파일의 열을 맞추어 병합
            all_data = pd.concat([all_data, df], ignore_index=True, sort=False)

    except Exception as e:
        print(f"파일을 읽는 중 오류 발생: {file}, 오류: {e}")

# 11. 병합된 데이터를 CSV 파일로 저장 (CP949 인코딩 적용)
csv_output_path = folder_path + "/신규P값병합08999.csv"

if not all_data.empty:
    # CSV 저장
    all_data.to_csv(csv_output_path, index=False, encoding='cp949')
    print(f"병합된 파일이 {csv_output_path}에 저장되었습니다.")
else:
    print("병합된 데이터가 비어 있습니다. 엑셀 파일들을 확인하세요.")

ValueError: mount failed